In [2]:
import random, os
import torch
import pandas as pd
import numpy as np
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
SEED = 42
DATA_PATH = "../data/"

In [4]:
train = pd.read_csv(f"{DATA_PATH}campaign_train.csv")
test = pd.read_csv(f"{DATA_PATH}campaign_test.csv")
train.shape, test.shape

((1344, 27), (896, 26))

In [5]:
train.head()

,ID,출생연도,고객_교육수준,고객_결혼여부,고객_소득,고객_자녀수,고객_청소년수,고객_가입날짜,고객_최신구매일_경과기간,고객_와인_구매금액,...,고객_카탈로그_통한_구매횟수,고객_매장방문_구매횟수,고객_지난달_회사사이트_방문횟수,캠페인1_수락여부,캠페인2_수락여부,캠페인3_수락여부,캠페인4_수락여부,캠페인5_수락여부,불만제기,target
0,train_0,1969,박사,기혼,30396.0,1,0,2014-04-30,22,15,...,1,2,7,0,0,1,0,0,0,1
1,train_1,1967,박사,기혼,36947.0,1,1,2012-08-07,49,88,...,0,4,9,0,0,0,0,0,0,0
2,train_2,1949,석사,사별,47570.0,1,1,2013-05-29,3,67,...,2,2,7,0,0,0,0,0,0,1
3,train_3,1976,석사,기혼,81929.0,1,0,2012-09-29,60,1486,...,4,10,6,0,0,1,0,1,0,1
4,train_4,1982,고졸,기혼,57937.0,0,1,2014-02-16,56,261,...,4,9,3,0,0,0,0,0,0,0


In [6]:
train_ft = train.iloc[:, 1:-1].copy()
test_ft = test.iloc[:, 1:].copy()

train_ft.shape , test_ft.shape

((1344, 25), (896, 25))

# 결측치 처리

In [7]:
mask = train_ft.isnull().sum() > 0
cols = train_ft.isnull().sum()[mask].index
cols

Index(['고객_교육수준', '고객_결혼여부', '고객_소득'], dtype='object')

In [8]:
e_level, m_status  = train_ft["고객_교육수준"].mode()[0], train_ft["고객_결혼여부"].mode()[0]
s_mean = train_ft["고객_소득"].mean()

In [9]:
train_ft["고객_교육수준"] = train_ft["고객_교육수준"].fillna(e_level)
train_ft["고객_결혼여부"] = train_ft["고객_결혼여부"].fillna(m_status)
train_ft["고객_소득"] = train_ft["고객_소득"].fillna(s_mean)

test_ft["고객_교육수준"] = test_ft["고객_교육수준"].fillna(e_level)
test_ft["고객_결혼여부"] = test_ft["고객_결혼여부"].fillna(m_status)
test_ft["고객_소득"] = test_ft["고객_소득"].fillna(s_mean)

In [10]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [11]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [12]:
train_ft = train_ft.drop(columns=["고객_가입날짜"])
test_ft = test_ft.drop(columns=["고객_가입날짜"])

In [13]:
cols = train_ft.select_dtypes("object").columns
cols

Index(['고객_교육수준', '고객_결혼여부'], dtype='object')

In [14]:
enc = OneHotEncoder(handle_unknown="ignore")
train_ft[enc.get_feature_names_out()] = enc.fit_transform(train_ft[cols]).A

In [15]:
test_ft[enc.get_feature_names_out()] = enc.transform(test_ft[cols]).A

In [16]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((1344, 32), (896, 32))

In [17]:
scaler = MinMaxScaler()
train_ft = scaler.fit_transform(train_ft)
test_ft = scaler.transform(test_ft)

In [18]:
train_ft

array([[0.71875   , 0.04311092, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.69791667, 0.05296299, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.51041667, 0.06893897, 0.5       , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.69791667, 0.05069811, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.86458333, 0.05272688, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.84375   , 0.05787174, 0.5       , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
target = train["target"].to_numpy().reshape(-1, 1).astype("float32")
target.shape

(1344, 1)

In [20]:
from datasets import Dataset

In [21]:
train_dict = {"x": train_ft, "label": target}
train_dt = Dataset.from_dict(train_dict)
train_dt

Dataset({
    features: ['x', 'label'],
    num_rows: 1344
})

In [22]:
train_dt.select([0,1,2])

Dataset({
    features: ['x', 'label'],
    num_rows: 3
})

# 모델 클래스

In [23]:
class Net(torch.nn.Module):
    def __init__(self, in_features, hidden_size=16):
        super().__init__()
        self.seq = torch.nn.Sequential(
            torch.nn.Linear(in_features, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(hidden_size, hidden_size // 2),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size // 2, 1),
        )
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

    def forward(self, x, labels=None):
        result = {"logits": self.seq(x)} # 예측값

        if labels is not None:
            result["loss"] = self.loss_fn(result["logits"], labels)

        return result

In [24]:
x = torch.Tensor(train_ft[:2])
x.shape

torch.Size([2, 32])

In [25]:
model = Net(train_ft.shape[1])
model(x, torch.Tensor(target[:2]))

{'logits': tensor([[-0.0416],
         [-0.0211]], grad_fn=<AddmmBackward0>),
 'loss': tensor(0.6984, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)}

# compute_metrics 파라미터에 전달할 콜백함수

In [26]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    true = eval_pred.label_ids
    pred = eval_pred.predictions
    sig = torch.nn.Sigmoid()
    pred = sig(torch.Tensor(pred)).numpy()
    return {"auc": roc_auc_score(true, pred)}

In [27]:
from transformers import TrainingArguments, Trainer

In [32]:
train_args_params = {
    "output_dir": "../output",
    "per_device_train_batch_size": 32,
    "per_device_eval_batch_size": 32,
    "learning_rate": 0.001,
    "num_train_epochs": 100,
    "lr_scheduler_type": "reduce_lr_on_plateau",
    "eval_strategy": "epoch",
    "logging_strategy": "epoch",
    "save_strategy": "epoch",
    "load_best_model_at_end": True,
    "metric_for_best_model": "auc",
    "greater_is_better": True,
    "report_to": "none",
}

train_args = TrainingArguments(**train_args_params)

In [33]:
from sklearn.model_selection import KFold

cv = KFold(5, shuffle=True, random_state=SEED)

In [34]:
from transformers import EarlyStoppingCallback

es_cb = EarlyStoppingCallback(10)

In [35]:
reset_seeds(SEED)
trainer_list = []

for i, (tri, vai) in enumerate(cv.split(train_ft)):
    x_train = train_dt.select(tri)
    x_valid = train_dt.select(vai)
    model = Net(train_ft.shape[1])
    trainer = Trainer(
        model,
        train_args,
        train_dataset=x_train,
        eval_dataset=x_valid,
        compute_metrics=compute_metrics,
        callbacks=[es_cb],
    )
    trainer.train()
    trainer_list.append(trainer)

  0%|          | 0/3400 [00:00<?, ?it/s]

{'loss': 0.7966, 'grad_norm': 0.48407790064811707, 'learning_rate': 0.001, 'epoch': 1.0}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.7443380355834961, 'eval_auc': 0.5346508330379298, 'eval_runtime': 0.029, 'eval_samples_per_second': 9275.924, 'eval_steps_per_second': 310.347, 'epoch': 1.0}
{'loss': 0.7091, 'grad_norm': 0.5499278903007507, 'learning_rate': 0.001, 'epoch': 2.0}


  0%|          | 0/9 [00:00<?, ?it/s]

...

{'eval_loss': 0.2385524958372116, 'eval_auc': 0.8847743165924984, 'eval_runtime': 0.028, 'eval_samples_per_second': 9571.878, 'eval_steps_per_second': 321.444, 'epoch': 48.0}
{'loss': 0.3349, 'grad_norm': 0.2811409831047058, 'learning_rate': 1.0000000000000002e-07, 'epoch': 49.0}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.2385532110929489, 'eval_auc': 0.8847743165924985, 'eval_runtime': 0.026, 'eval_samples_per_second': 10307.683, 'eval_steps_per_second': 346.154, 'epoch': 49.0}
{'loss': 0.3377, 'grad_norm': 0.3841450810432434, 'learning_rate': 1.0000000000000002e-07, 'epoch': 50.0}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.23855243623256683, 'eval_auc': 0.8847743165924985, 'eval_runtime': 0.027, 'eval_samples_per_second': 9925.769, 'eval_steps_per_second': 333.328, 'epoch': 50.0}
{'train_runtime': 9.772, 'train_samples_per_second': 11011.105, 'train_steps_per_second': 347.935, 'train_loss': 0.3535710003796746, 'epoch': 50.0}


In [36]:
scores = []
for trainer in trainer_list:
    scores.append(trainer.state.best_metric)

np.mean(scores)

0.8318624510437662

# 테스트 데이터 예측

In [37]:
test_dict = {"x": test_ft}
test_dt = Dataset.from_dict(test_dict)
test_dt

Dataset({
    features: ['x'],
    num_rows: 896
})

In [38]:
sig = torch.nn.Sigmoid()
pred_list = []
for trainer in trainer_list:
    pred_output = trainer.predict(test_dt)
    pred = torch.Tensor(pred_output.predictions)
    pred = sig(pred).numpy()
    pred_list.append(pred)

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [39]:
pred = np.mean(pred_list, axis=0)
pred

array([[0.05044533],
       [0.09840681],
       [0.0909398 ],
       [0.33907896],
       [0.09959547],
       [0.3112548 ],
       [0.5630649 ],
       [0.14410111],
       [0.1128155 ],
       [0.07603491],
       [0.07350561],
       [0.0881794 ],
       [0.1505562 ],
       [0.08533502],
       [0.38068622],
       [0.05456564],
       [0.12584177],
       [0.49073964],
       [0.05581699],
       [0.16177419],
       [0.10844004],
       [0.17539969],
       [0.03216618],
       [0.05652187],
       [0.06908792],
       [0.2228659 ],
       [0.10119226],
       [0.12001918],
       [0.1363337 ],
       [0.21725789],
       [0.1224153 ],
       [0.26481122],
       [0.08616831],
       [0.4805251 ],
       [0.08482625],
       [0.0258268 ],
       [0.38198194],
       [0.06233369],
       [0.06362274],
       [0.0603743 ],
       [0.41983086],
       [0.03229585],
       [0.18673691],
       [0.03786109],
       [0.05390849],
       [0.02575592],
       [0.04876714],
       [0.048